In [1]:
import pandas as pd
import numpy as np
import cv2

In [5]:
common_path ="pilot_1/"

In [6]:
# Function to map gaze positions to pixel coordinates
def get_row_number(eye_tracking_data, des_gidx):
    if des_gidx in eye_tracking_data['gidx'].values:
        row_number = eye_tracking_data.index[eye_tracking_data['gidx'] == des_gidx].tolist()
        print('Exist')
        return row_number[0] if row_number else None
    else:
        return None
    
def map_gaze_to_pixels(gaze_position_x,gaze_position_y, frame_width, frame_height):
    if gaze_position_x is not None and gaze_position_y is not None:
        mapped_x = int(gaze_position_x * frame_width)
        mapped_y = int(gaze_position_y * frame_height)
        return mapped_x, mapped_y
    return None

def overlay_gaze_on_frame(frame, des_row,eye_tracking_data,frame_width,frame_height):
    last_row_number = eye_tracking_data.shape[0] - 1
    # print(des_row)
    for i in range(4):
        if (des_row+i) <= last_row_number:
            gaze_position_x = eye_tracking_data['gaze_position_x'].iloc[des_row+i]
            gaze_position_y = eye_tracking_data['gaze_position_y'].iloc[des_row+i]
            print(f'row:{des_row}')

            # Map gaze point to pixel coordinates
            mapped_gaze_position = map_gaze_to_pixels(gaze_position_x,gaze_position_y, frame_width, frame_height)
            if mapped_gaze_position is not None:
                x, y = mapped_gaze_position
                if x !=0 and y!=0:
                    cv2.circle(frame, (int(x), int(y)), 10, (0, 0, 255), -1)  # Red color, larger radius
        else:
            break
    return frame

In [7]:

# Video file path
video_file_path = f'{common_path}fullstream.mp4'
eye_tracking_file_path = f'{common_path}data.csv'  # Replace with the actual gaze data file

# Read video frames
cap = cv2.VideoCapture(video_file_path)
if not cap.isOpened():
    print("Error: Couldn't open the video file.")
else:
    # Read gaze data
    eye_tracking_data = pd.read_csv(eye_tracking_file_path,delimiter=',')
    # Get video details
    width  = int(cap.get(3))
    height = int(cap.get(4))
    fps    = cap.get(5)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(f'{common_path}gaze_mapping_video.mp4', fourcc, fps, (width, height))

    first_gidx = eye_tracking_data['gidx'].iloc[0]
    # Process video frames
    for frame_count in range(int(cap.get(7))):  # Total number of frames
        ret, frame = cap.read()

        # Break the loop if the video has ended
        if not ret:
            break

        # Check if gaze data is available for the current frame
        des_row = get_row_number(eye_tracking_data,(frame_count*4 +first_gidx))  # Assuming frame_count starts from 1
        if des_row is not None:
            # Overlay mapped gaze point on the frame
            frame_with_gaze = overlay_gaze_on_frame(frame.copy(),des_row,eye_tracking_data,width,height)

        # Write the frame to the output video
        out.write(frame_with_gaze)

    # Release the video capture and writer objects
    cap.release()
    out.release()

    # Close all windows
    cv2.destroyAllWindows()

    print("Output video saved as 'gaze_mapping_video.mp4'")

Exist
row:0
row:0
row:0
row:0
Exist
row:4
row:4
row:4
row:4
Exist
row:8
row:8
row:8
row:8
Exist
row:12
row:12
row:12
row:12
Exist
row:16
row:16
row:16
row:16
Exist
row:20
row:20
row:20
row:20
Exist
row:24
row:24
row:24
row:24
Exist
row:28
row:28
row:28
row:28
Exist
row:32
row:32
row:32
row:32
Exist
row:36
row:36
row:36
row:36
Exist
row:40
row:40
row:40
row:40
Exist
row:44
row:44
row:44
row:44
Exist
row:48
row:48
row:48
row:48
Exist
row:52
row:52
row:52
row:52
Exist
row:56
row:56
row:56
row:56
Exist
row:60
row:60
row:60
row:60
Exist
row:66
row:66
row:66
row:66
Exist
row:70
row:70
row:70
row:70
Exist
row:74
row:74
row:74
row:74
Exist
row:77
row:77
row:77
row:77
Exist
row:81
row:81
row:81
row:81
Exist
row:85
row:85
row:85
row:85
Exist
row:89
row:89
row:89
row:89
Exist
row:93
row:93
row:93
row:93
Exist
row:95
row:95
row:95
row:95
Exist
row:99
row:99
row:99
row:99
Exist
row:103
row:103
row:103
row:103
Exist
row:107
row:107
row:107
row:107
Exist
row:111
row:111
row:111
row:111
Exist
row:115
